In [ ]:
import random
import math
from abc import ABC, abstractmethod

Remember that each node, is a logistic regression in and out of itself


In [ ]:
class NodeBlueprint(ABC):
    @abstractmethod
    def __init__(self, input_features_num:int,activation:str):
        pass
    @abstractmethod    
    def forward(self, input:list):
        pass
    @abstractmethod
    def backward(self, input:list, error:float, learning_rate:float):
        pass
    
    @staticmethod
    def sigmoid(x:float):
        return 1/(1+math.exp(-x))
    
    @staticmethod
    def relu(x:float):
        return max(0,x)
    
    @staticmethod
    def tanh(x:float):
        return math.tanh(x)

In [ ]:
import random

class Node:
    def __init__(self, input_features_num: int, activation: str):
        self.weights = [random.random() for _ in range(input_features_num)]
        self.bias = random.random()
        self.activation = self.sigmoid if activation == "sigmoid" else self.relu
        
        self.input_cache = []
        self.z_cache = []
        self.activation_cache = []
    
    def sigmoid(self, x):
        return 1 / (1 + math.exp(-x))
    
    def relu(self, x):
        return max(0, x)

    def forward(self, layer_input: list):
        assert len(layer_input) == len(self.weights), 'Number of weights must be equal to number of input features'
        z = sum(i*w for i, w in zip(layer_input, self.weights)) + self.bias
        a = self.activation(z)
        
        self.input_cache=layer_input.copy()
        self.z_cache=z
        self.activation_cache=a
        
        return a
    
    def backward(self, output_gradient: float, learning_rate: float):
        if self.activation == self.sigmoid:
            d_activation = self.activation_cache * (1 - self.activation_cache)
        elif self.activation == self.relu:
            d_activation = 1 if self.z_cache > 0 else 0
        
        d_z = output_gradient * d_activation
        d_w = [d_z * i for i in self.input_cache]
        d_b = d_z
        
        # Update weights and bias
        self.weights = [w - learning_rate * dw for w, dw in zip(self.weights, d_w)]
        self.bias -= learning_rate * d_b
        
        # Return gradient w.r.t. input for further backpropagation in the network
        input_gradient = [d_z * w for w in self.weights]
        return input_gradient
        
        
        


In [ ]:
class DenseLayer():
    def __init__(self, input_features_num:int, nodes_num:int, activation:str):
        self.activation = activation
        self.nodes = [Node(input_features_num,activation) for _ in range(nodes_num)]
    
    def forward(self, input:list):
        return [node.forward(input) for node in self.nodes]
    
    def backward(self, input:list, error:list, learning_rate:float):
        for node, err in zip(self.nodes,error):
            node.backward(input,err,learning_rate)